In [1]:
import os
from llama_index.indices.struct_store import GPTPandasIndex, GPTNLPandasQueryEngine
from get_links import get_links
import pandas as pd

os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:10809"
os.environ["OPENAI_API_KEY"] = 'sk-xxxxx'

[('', '/'), ('Qzone', 'https://qzone.qq.com'), ('点击查看QQ空间', 'https://qzone.qq.com'), ('Qmail', 'https://mail.qq.com'), ('点击查看QQ邮箱', 'https://mail.qq.com/cgi-bin/loginpage'), ('登录', 'javascript:;'), ('用户协议', '//www.qq.com/contract.shtml'), ('隐私条款', '//privacy.qq.com/policy/cookie-privacypolicy'), ('[退出登录]', 'javascript:;'), ('关怀版', '//new.qq.com/barrierfree.html'), ('无障碍浏览', 'javascript:void(0)'), ('新闻', 'http://news.qq.com/'), ('财经', 'https://new.qq.com/ch/finance/'), ('科技', 'https://new.qq.com/ch/tech/'), ('娱乐', 'https://new.qq.com/ch/ent/'), ('体育', 'https://sports.qq.com/'), ('NBA', 'https://sports.qq.com/nba/'), ('国际', 'https://new.qq.com/ch/world/'), ('军事', 'https://new.qq.com/ch/milite/'), ('文化', 'https://new.qq.com/ch/cul/'), ('教育', 'https://new.qq.com/ch/edu/'), ('健康', 'https://new.qq.com/ch/health/'), ('汽车', 'https://new.qq.com/ch/auto/'), ('房产', 'http://house.qq.com/'), ('游戏', 'https://new.qq.com/ch/games/'), ('乐活', 'https://new.qq.com/ch/fashion/'), ('云游戏', 'https://gamer.qq.

In [2]:
df = pd.DataFrame(get_links("https://www.qq.com/"),columns=["网页标题","网页链接"])
df.drop_duplicates(subset=['网页标题'],keep='first',inplace=True)
df.reset_index(drop=True,inplace=True)

In [3]:
df

,网页标题,网页链接
0,,/
1,Qzone,https://qzone.qq.com
2,点击查看QQ空间,https://qzone.qq.com
3,Qmail,https://mail.qq.com
4,点击查看QQ邮箱,https://mail.qq.com/cgi-bin/loginpage
...,...,...
206,工商网监电子标识,http://szcert.ebs.org.cn/6917b6fe-b844-4e12-97...
207,深圳市\r 消费者委员会,http://www.sz315.org
208,刷新,javascript:
209,用户反馈,https://support.qq.com/products/4312


In [4]:
index = GPTPandasIndex(df)
index_nlp = GPTNLPandasQueryEngine(index)

In [5]:
response = index_nlp.query(
    "帮我找找这个网页上的股票入口",
)

In [6]:
print(response.response)


df[df['网页标题'].str.contains('股票')]['网页链接']


In [7]:
search_link = eval(response.response)
search_link.item()

'https://new.qq.com/ch/finance_stock/'

## 查到了入口链接

In [8]:
from llama_index import GPTListIndex, TrafilaturaWebReader
documents = TrafilaturaWebReader().load_data([search_link.item()])
index = GPTListIndex.from_documents(documents)
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("上证指数怎么样？”")

In [9]:
response.response

'\nThe Shanghai Composite Index is currently trading at 3,400.45, up 0.14% from the previous close.'

In [10]:
from llama_index import GPTListIndex, TrafilaturaWebReader
documents = TrafilaturaWebReader().load_data(["https://finance.sina.com.cn/stock/"])
index = GPTListIndex.from_documents(documents)
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("这个网页有巴菲特的消息吗？如果有，标题是什么？")

Token indices sequence length is longer than the specified maximum sequence length for this model (2536 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
response.response

'\n\n是的，有巴菲特的消息。标题是：巴菲特再度减持比亚迪H股持股比例降至9.87%，涉及信披违规被调、去年归属净利润减少17.19%、何其金家族拟10亿入主会畅通讯、实控人失联、中金公司、首旅酒店、新华网等遭到北向资金净流入等，并受到市盈排行、分红融资、'